In [12]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving my_utils.py to my_utils (1).py


In [13]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [16]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr  2 11:32:14 2019
@author: olevi
"""

#%%
import pandas as pd
import numpy as np
import codecs
import nltk
import pickle
import my_utils
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer,CountVectorizer,HashingVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,roc_auc_score,log_loss,precision_score,recall_score
from sklearn.metrics.pairwise import linear_kernel

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier


from scipy.stats import pearsonr

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


#data_files = my_utils.read_fake_satire_data("FakeNewsData/StoryText 2/")
data_files = my_utils.read_fake_satire_data("drive/My Drive/Colab Notebooks/data/FakeNewsData/StoryText 2/")
headlines_data = data_files[0]
text_data = data_files[1]
target = data_files[2]

#%%
# SET TOKENIZER WITH STEMMING
# from nltk.stem.porter import *
# stemmer = PorterStemmer()
stemmer = SnowballStemmer("english")

# pattern = r'[\d.,]+|[A-Z][.A-Z]+\b\.*|\w+|\S'
pattern = r'\w+|\?|\!|\"|\'|\;|\:'


class Tokenizer(object):
    def __init__(self):
        self.tok = RegexpTokenizer(pattern)
        self.stemmer = stemmer

    def __call__(self, doc):
        return [self.stemmer.stem(token) 
                for token in self.tok.tokenize(doc)]

#%%  
# SET PIPELINE WITH TOKENIZER, N-GRAMS AND LOGISTIC REGRESSION MODEL
clf_text = Pipeline([('vect', TfidfVectorizer()),
                      #('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB())])
             

clf_text1 = Pipeline([('vect', TfidfVectorizer(tokenizer=Tokenizer(), stop_words='english', ngram_range=(1, 2), max_features=500)),
                      #('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB())])
             
#%%
# SAVE CLASSIFIER
filename = 'text_classifier.pk'
with open(''+filename, 'wb') as file:
	pickle.dump(clf_text, file)


#%%
# LOAD CLASSIFIER
with open("text_classifier.pk" ,'rb') as f:
    clf_text= pickle.load(f)

#%%
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf_text1, text_data, target, cv=10)  #scoring='roc_auc'
# parts that I added
r_scores = cross_val_score(clf_text1, text_data, target, cv=10, scoring='recall_macro')  #scoring='recall'
p_scores = cross_val_score(clf_text1, text_data, target, cv=10, scoring='precision_macro')  #scoring='precision'
f_scores = cross_val_score(clf_text1, text_data, target, cv=10, scoring='f1_macro')  #scoring='f1'

print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (r_scores.mean(), r_scores.std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (p_scores.mean(), p_scores.std() * 2))
print("F1: %0.2f (+/- %0.2f)" % (f_scores.mean(), f_scores.std() * 2))

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/data/FakeNewsData/StoryText 2/Fake/finalFake/222.txt'
[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/data/FakeNewsData/StoryText 2/Fake/finalFake/223.txt'


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.6/dist-packages/sklearn/

Accuracy: 0.70 (+/- 0.14)
Recall: 0.51 (+/- 0.28)
Precision: 0.69 (+/- 0.24)
F1: 0.58 (+/- 0.25)
